## Anomaly Detection in 1d Time Series

### Prepare Data

In [ ]:
# metric is the column in flowlogs_df where anomalies should be detected.
# sample_rate is the rate this column will be resampled and aggregated.
# train_size is the relative size of the training set.
# window_size is the length of each row in the shingled data.
metric = 'ABBytes', sample_rate = '1Min', train_size = 0.8, window_size = 16

In [ ]:
from data_preparation import time_series_1d, standardize, shingle

print("Preparing data for learning task...")
data_df = time_series_1d(flowlogs_df, metric, sample_rate)
split_index = int(train_size*len(data_df.index))
train_set, train_labels = shingle(standardize(data_df[:split_index]), window=window_size)
test_set, test_labels = shingle(standardize(data_df[split_index:]), window=window_size)
print("Done.")

### Learn and Predict

In [ ]:
epochs = 10   # Duration of training.

In [ ]:
from machine_learning import cnn_regression

pred = cnn_regression(train_set, test_set, train_labels, test_labels, max_epochs=epochs)

### Visualize Anomalies

In [ ]:
from anomaly_processing import find_anomalies, anomaly_visualization

actual = np.reshape(test_labels.values, len(test_labels.index))
scores = np.abs(actual - pred)
anomaly_indices = find_anomalies(scores)
anomaly_visualization(test_labels.index, test_labels.values, anomaly_indices)